## Imports

In [2]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import cifar10  #
from keras.models import Sequential  # Model type to be used
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
import keras.utils as np_utils

### import pre-computed curvatures

In [27]:
with open(f'cifar_output_curv_AVG', 'rb') as file:
  output_curvatures = np.array(pickle.load(file))

with open(f'cifar_input_curv', 'rb') as file:
  input_curvatures = pickle.load(file)

del file

## Define functions

In [18]:
def shuffle_data(X: np.ndarray, Y: np.ndarray, seed: int):
  np.random.seed(seed)
  np.random.shuffle(X)
  np.random.seed(seed)
  np.random.shuffle(Y)

In [19]:
def create_cifar_model():
  return Sequential([
    Conv2D(32, 3, padding='same', input_shape=cifar_X_train.shape[1:], activation='relu'),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax'),
  ])

In [35]:
def get_reduced_datasets(data_size: int, X: np.ndarray, Y: np.ndarray, order_method: str, curvature_set='input'):

  _input_curvatures = input_curvatures
  _output_curvatures = output_curvatures
  _train_y = cifar_train_y[:, 0]

  _curvature_set = _input_curvatures if curvature_set == 'input' else _output_curvatures

  for y_class in range(10):
    class_indices = _train_y == y_class
    keep_indices = None
    if order_method == 'random':
      keep_indices = np.random.choice(X[class_indices].shape[0], data_size, replace=True)
    elif order_method == 'low_to_high':
      keep_indices = np.argsort(_curvature_set[y_class][:, 0])[:data_size]
    elif order_method == 'high_to_low':
      keep_indices = np.argsort(_curvature_set[y_class])[-data_size:]
    elif order_method == 'mid':
      num_low_curv = data_size // 2
      num_high_curv = data_size - num_low_curv

      curv_midpoint = _curvature_set[y_class][:, 0].shape[0] // 2

      keep_indices = np.argsort(_curvature_set[y_class])[curv_midpoint - num_low_curv : curv_midpoint + num_high_curv]
    elif order_method == 'ratio_low_to_high':
      ratios = _output_curvatures[y_class][:, 0] / _input_curvatures[y_class][:, 0]
      keep_indices = np.argsort(ratios)[:data_size]
    elif order_method == 'ratio_high_to_low':
      ratios = _output_curvatures[y_class][:, 0] / _input_curvatures[y_class][:, 0]
      keep_indices = np.argsort(ratios)[-data_size:]
    else:
      raise ValueError(f"order method not implemented: {order_method}")

    new_x_row = X[class_indices][keep_indices]
    new_y_row = Y[class_indices][keep_indices]

    del keep_indices

    Reduced_X = np.vstack([Reduced_X, new_x_row]) if y_class > 0 else new_x_row
    Reduced_Y = np.vstack([Reduced_Y, new_y_row]) if y_class > 0 else new_y_row

  return Reduced_X, Reduced_Y

In [21]:
def get_accuracies(X_train, Y_train, X_test, Y_test, order_method='low_to_high', curvature_set='input', num_models=5, shuffle_seed=None):
  valacclist = []
  acclist = []
  for data_size in datasizes:
    Reduced_X_train, Reduced_Y_train = get_reduced_datasets(data_size, X_train, Y_train, order_method, curvature_set)
    print(f'ReducedX.shape: {Reduced_X_train.shape}')

    if shuffle_seed is not None:
      shuffle_data(Reduced_X_train, Reduced_Y_train, shuffle_seed)

    valacclist.append([])
    acclist.append([])
    for _ in range(num_models):
      nt = create_cifar_model()
      nt.compile(loss= tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['categorical_accuracy'])
      history  = nt.fit(Reduced_X_train, Reduced_Y_train, epochs=20, validation_data=(X_test, Y_test), verbose=0, batch_size = 128)
      del nt
      print(f"  train acc = {history.history['categorical_accuracy'][-1]} val acc = {history.history['val_categorical_accuracy'][-1]}")
      valacclist[-1].append(history.history['val_categorical_accuracy'][-1])
      acclist[-1].append(history.history['categorical_accuracy'][-1])
      del history

    del Reduced_X_train, Reduced_Y_train
  return valacclist, acclist

In [22]:
def plot_accuracies(accuracies, names=None):
  plt.figure(figsize=(14, 10))
  for acclist in accuracies:
    plt.plot(datasizes, np.mean(acclist, axis=1))

  if names is not None:
    plt.legend(names, fontsize=11)

  plt.xscale('log')
  plt.gca().invert_xaxis()
  plt.grid()
  plt.show()

# cifar

In [23]:
(cifar_X_train, cifar_train_y), (cifar_X_test, cifar_test_y) = cifar10.load_data()
cifar_X_train = cifar_X_train / 255
cifar_X_test = cifar_X_test/ 255

In [24]:
cifar_Y_train = np_utils.to_categorical(cifar_train_y, 10)
cifar_Y_test = np_utils.to_categorical(cifar_test_y, 10)

### fine grain analysis on best performers

In [5]:
datasizes = np.linspace(50000, 30000, 15).astype('int')

In [4]:
print(datasizes)

[5000 3773 2848 2149 1622 1224  923  697  526  397]


In [6]:
print(datasizes)

[50000 48571 47142 45714 44285 42857 41428 40000 38571 37142 35714 34285
 32857 31428 30000]


In [ ]:
rand, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='random', num_models=5)

In [ ]:
high, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='high_to_low', curvature_set='output', shuffle_seed=1337, num_models=10)

In [ ]:
ratio_low, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='ratio_low_to_high', shuffle_seed=1337, num_models=10)

In [ ]:
ratio_high, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='ratio_high_to_low', shuffle_seed=1337, num_models=10)

In [ ]:
namelist = ['Random selection', 'Highest Curvatures', 'Low Output/Input Ratio', 'High Output/Input Ratio']
plot_accuracies([rand, high, ratio_low, ratio_high], namelist)

### close up on ratio tests

In [ ]:
namelist = ['Random selection', 'Low Output/Input Ratio', 'High Output/Input Ratio']
datasizes = np.logspace(3.699, 2.599, 20).astype('int')

rand, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='random', num_models=15)
ratio_low, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='ratio_low_to_high', shuffle_seed=1337, num_models=15)
ratio_high, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='ratio_high_to_low', shuffle_seed=1337, num_models=15)

plot_accuracies([rand, ratio_low, ratio_high], namelist)

In [ ]:
plot_accuracies([rand, ratio_low, ratio_high], namelist)

In [ ]:
# these take a while to train, so I'm breaking them into their own cells to avoid retraining all of them if changes need to be mande to just one
rand, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='random')

In [ ]:
hess, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='hessian')

In [ ]:
keep_high, _ = get_accuracies(cifar_X_train, cifar_Y_train, cifar_X_test, cifar_Y_test, order_method='high_to_low', curvature_set='output')

In [ ]:
plot_accuracies([rand, hess, keep_high], ['rand', 'hess', 'keep_high'])